In [0]:
import os
from dotenv import load_dotenv
import psycopg2
from pyspark.sql.utils import AnalysisException

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

# --- CONFIGURAÇÃO DA CONEXÃO ---
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")

# Validação para garantir que as variáveis foram carregadas
if not all([db_user, db_password, db_host, db_port, db_name]):
    raise ValueError("Uma ou mais variáveis de ambiente do banco de dados não foram definidas. Verifique o arquivo .env")

jdbc_url = f"jdbc:postgresql://{db_host}:{db_port}/{db_name}"

connection_properties = {
    "user": db_user,
    "password": db_password,
    "driver": "org.postgresql.Driver"
}

# --- DEFINIÇÃO DOS CAMINHOS ---
gold_base_path = "/mnt/datalake/gold/"
control_table_name = "destiny._load_control"

# Tabelas da camada Gold para carregar no banco
gold_tables_to_load = [
    "dim_companies", "dim_locations", "dim_skills", "dim_date", 
    "fact_jobs", "fact_job_skills"
]

# --- FUNÇÕES DE CONTROLE ---

def initialize_control_table():
    """Cria a tabela de controle se ela não existir."""
    try:
        conn = psycopg2.connect(host=db_host, dbname=db_name, user=db_user, password=db_password, port=db_port)
        cursor = conn.cursor()
        
        # Cria o esquema 'destiny' se não existir
        cursor.execute("CREATE SCHEMA IF NOT EXISTS destiny;")

        # Cria a tabela de controle
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {control_table_name} (
            table_name VARCHAR(255) PRIMARY KEY,
            last_load_timestamp TIMESTAMP,
            status VARCHAR(50)
        );
        """
        cursor.execute(create_table_query)
        conn.commit()
        cursor.close()
        conn.close()
        print(f"Tabela de controle '{control_table_name}' verificada/criada com sucesso.")
    except Exception as e:
        print(f"Erro ao inicializar a tabela de controle: {e}")
        raise e

def check_if_loaded(table_name):
    """Verifica na tabela de controle se os dados já foram carregados com sucesso."""
    try:
        # Tenta ler a contagem da tabela de controle
        query = f"(SELECT COUNT(*) FROM {control_table_name} WHERE table_name = '{table_name}' AND status = 'SUCCESS') as t"
        count_df = spark.read.jdbc(url=jdbc_url, table=query, properties=connection_properties)
        return count_df.first()[0] > 0
    except AnalysisException as e:
        # Se a tabela de controle não existe, retorna False (não foi carregado)
        if "relation" in str(e) and "does not exist" in str(e):
            return False
        raise e

def update_load_control(table_name, status):
    """Atualiza o status de carga na tabela de controle."""
    try:
        conn = psycopg2.connect(host=db_host, dbname=db_name, user=db_user, password=db_password, port=db_port)
        cursor = conn.cursor()
        
        upsert_query = f"""
        INSERT INTO {control_table_name} (table_name, last_load_timestamp, status)
        VALUES (%s, NOW(), %s)
        ON CONFLICT (table_name) 
        DO UPDATE SET 
            last_load_timestamp = NOW(),
            status = %s;
        """
        cursor.execute(upsert_query, (table_name, status, status))
        conn.commit()
        cursor.close()
        conn.close()
        print(f"Controle de carga atualizado para '{table_name}' com status '{status}'.")
    except Exception as e:
        print(f"Erro ao atualizar o controle de carga: {e}")


# --- PROCESSO DE CARGA ---

print("Iniciando carga da camada Gold para o banco de dados...")

# 1. Garante que a tabela de controle exista
initialize_control_table()

# 2. Loop para carregar cada tabela
for table_name in gold_tables_to_load:
    # Verifica se já foi carregada
    if check_if_loaded(table_name):
        print(f"Tabela '{table_name}' já foi carregada com sucesso. Pulando.")
        continue

    try:
        print(f"Iniciando carga da tabela '{table_name}'...")
        
        # Carrega os dados da camada Gold
        df_gold = spark.read.format("delta").load(f"{gold_base_path}{table_name}")
        
        # Define o nome da tabela de destino no esquema 'destiny'
        destiny_table_name = f"destiny.{table_name}"
        
        df_gold.write.jdbc(
            url=jdbc_url,
            table=destiny_table_name,
            mode="overwrite", 
            properties=connection_properties
        )
        
        print(f"Tabela '{table_name}' carregada com sucesso no banco de dados.")
        
        # Atualiza a tabela de controle com o status de sucesso
        update_load_control(table_name, "SUCCESS")

    except Exception as e:
        print(f"ERRO ao carregar a tabela '{table_name}': {e}")
        # Atualiza a tabela de controle com o status de falha
        update_load_control(table_name, "FAILED")
        break

print("Processo de carga para o banco de dados finalizado.")